<a href="https://colab.research.google.com/github/angad68/topsisForPretrainedModels/blob/main/topsisForPretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Dataset**

In [6]:
import numpy as np
import pandas as pd

# Example data for Text Summarization models
# Criteria: ROUGE, Inference Time, Model Size, F1 Score
data = {
    "Model": ["BART", "T5", "Pegasus", "DistilBART"],
    "ROUGE": [0.85, 0.88, 0.86, 0.83],
    "Inference Time (ms)": [120, 150, 110, 100],
    "Model Size (MB)": [500, 400, 600, 350],
    "F1 Score": [0.87, 0.89, 0.88, 0.84]
}

# Convert data to DataFrame
df = pd.DataFrame(data)
print("Initial Data:")
print(df)

Initial Data:
        Model  ROUGE  Inference Time (ms)  Model Size (MB)  F1 Score
0        BART   0.85                  120              500      0.87
1          T5   0.88                  150              400      0.89
2     Pegasus   0.86                  110              600      0.88
3  DistilBART   0.83                  100              350      0.84


**Normalization**

In [7]:
def normalize(df, benefit_criteria):
    norm_df = df.copy()
    for col in df.columns[1:]:  # Exclude model names
        if col in benefit_criteria:
            norm_df[col] = df[col] / np.sqrt(np.sum(df[col] ** 2))  # Beneficial criteria
        else:
            norm_df[col] = np.sqrt(np.sum(df[col] ** 2)) / df[col]  # Non-beneficial criteria
    return norm_df

**Weight definition**

In [8]:
# Step 2: Define weights for the criteria
# ROUGE: 0.4, Inference Time: 0.2, Model Size: 0.2, F1 Score: 0.2
weights = np.array([0.4, 0.2, 0.2, 0.2])
benefit_criteria = ["ROUGE", "F1 Score"]  # Criteria where higher is better

Nromalization

In [9]:
normalized_df = normalize(df.drop(columns=["Model"]), benefit_criteria)
print("\nNormalized Data:")
print(normalized_df)
weighted_df = normalized_df * weights
print("\nWeighted Normalized Data:")
print(weighted_df)



Normalized Data:
   ROUGE  Inference Time (ms)  Model Size (MB)  F1 Score
0   0.85             2.024160         1.889444  0.499884
1   0.88             1.619328         2.361805  0.511376
2   0.86             2.208174         1.574537  0.505630
3   0.83             2.428992         2.699206  0.482647

Weighted Normalized Data:
   ROUGE  Inference Time (ms)  Model Size (MB)  F1 Score
0  0.340             0.404832         0.377889  0.099977
1  0.352             0.323866         0.472361  0.102275
2  0.344             0.441635         0.314907  0.101126
3  0.332             0.485798         0.539841  0.096529


**Ideal Best/Worst -- Column Creation**

In [11]:
ideal_best = weighted_df.max()
ideal_worst = weighted_df.min()

distance_to_best = np.sqrt(((weighted_df - ideal_best) ** 2).sum(axis=1))
distance_to_worst = np.sqrt(((weighted_df - ideal_worst) ** 2).sum(axis=1))

closeness = distance_to_worst / (distance_to_best + distance_to_worst)



Final Ranking:
        Model  ROUGE  Inference Time (ms)  Model Size (MB)  F1 Score  \
3  DistilBART   0.83                  100              350      0.84   
1          T5   0.88                  150              400      0.89   
0        BART   0.85                  120              500      0.87   
2     Pegasus   0.86                  110              600      0.88   

   TOPSIS Score  Rank  
3      0.930164   1.0  
1      0.475157   2.0  
0      0.361951   3.0  
2      0.340584   4.0  


In [12]:

df["TOPSIS Score"] = closeness
df["Rank"] = df["TOPSIS Score"].rank(ascending=False)

print("\nFinal Ranking:")
print(df.sort_values("Rank"))



Final Ranking:
        Model  ROUGE  Inference Time (ms)  Model Size (MB)  F1 Score  \
3  DistilBART   0.83                  100              350      0.84   
1          T5   0.88                  150              400      0.89   
0        BART   0.85                  120              500      0.87   
2     Pegasus   0.86                  110              600      0.88   

   TOPSIS Score  Rank  
3      0.930164   1.0  
1      0.475157   2.0  
0      0.361951   3.0  
2      0.340584   4.0  
